## F319 Posts Aggregation

In [7]:
import pandas as pd
import re
from tqdm import tqdm
import os

# Regex pattern to remove everything from "XYZ đã viết:" to "xem tất cả"
reply_block_pattern = re.compile(r".*? đã viết:.*?xem tất cả\.?", flags=re.DOTALL | re.IGNORECASE)

def clean_reply(text):
    if not text or not isinstance(text, str):
        return text

    # Find the start of the quoted reply
    start_idx = text.find(" đã viết:")
    if start_idx == -1:
        return text.strip()  # no reply block, keep text

    # Find the end of "xem tất cả"
    end_idx = text.lower().find("xem tất cả", start_idx)
    if end_idx == -1:
        return text[:start_idx].strip()  # cut everything from start_idx to end
    else:
        return text[end_idx + len("xem tất cả"):].strip()

def preprocess_f319_posts(file_paths, output_path='f319_data/aggregated_f319_data/aggregated_f319_posts.csv'):
    """
    Preprocess F319 posts using pandas:
    - Reads multiple CSV files
    - Converts post_time to datetime
    - Removes quoted reply blocks ("XYZ đã viết:\n... Xem tất cả")
    - Keeps one row per post
    - Filters out posts before 2020-01-01
    - Saves the combined cleaned DataFrame to CSV
    
    Args:
        file_paths: list of CSV file paths to read
        output_path: path to save the final CSV
    
    Returns:
        A single pandas DataFrame with columns ['topic_title', 'date', 'content']
    """
    dfs = []

    for path in tqdm(file_paths, desc="Processing F319 post files"):
        df = pd.read_csv(path, encoding="utf-8", dtype=str, usecols=['topic_title', 'post_time', 'content'])
        
        # Drop rows with missing essential columns
        df = df.dropna(subset=['topic_title', 'post_time', 'content'])

        # Convert post_time to datetime
        df['date'] = pd.to_datetime(df['post_time'].str.split(',').str[0], format='%d/%m/%Y', errors='coerce')
        df = df.drop(columns=['post_time'])
        

        df['content'] = df['content'].apply(clean_reply)

        dfs.append(df)

    # Combine all files into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    # Filter out posts before 2020-01-01
    combined_df = combined_df[combined_df['date'] > pd.Timestamp("2020-01-01")]

    # Make sure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save to CSV
    combined_df.to_csv(output_path, index=False)
    print(f"Saved aggregated posts to '{output_path}'")
    print(f"Total posts after filtering: {len(combined_df)}")

    return combined_df


In [9]:
file_paths = [f'f319_data/posts_{i:04d}.csv' for i in range(1, 514)]
preprocess_f319_posts(file_paths)

Processing F319 post files: 100%|██████████| 513/513 [00:48<00:00, 10.65it/s]


Saved aggregated posts to 'f319_data/aggregated_f319_data/aggregated_f319_posts.csv'
Total posts after filtering: 5457935


,topic_title,content,date
0,"Lạm phát và thị trường chứng khoán, mối liên h...","(Dữ liệu: thị trường Mỹ, 1926 - 2002. Hình từ ...",2025-10-26
1,"Lạm phát và thị trường chứng khoán, mối liên h...",Mới bơm tiền vài tháng. Ăn thua đ gì mà lo sớm...,2025-10-26
2,CỔ PHIẾU HLC – “VIÊN NGỌC ẨN” CỦA NGÀNH THAN,Công ty Cổ phần Than Hà Lầm – Vinacomin (\nHNX...,2025-10-23
3,CỔ PHIẾU HLC – “VIÊN NGỌC ẨN” CỦA NGÀNH THAN,"HLC quả rẻ bác, năm nay lãi to chia cổ tức tiề...",2025-10-24
4,CỔ PHIẾU HLC – “VIÊN NGỌC ẨN” CỦA NGÀNH THAN,"Vãi, sàn luôn.",2025-10-24
...,...,...,...
5915629,Triệu tập Hội Đồng Gió Mát - Tập 03,"Inbox gì mén\n--- Gộp bài viết,\n17/01/2020\n-...",2020-01-17
5915630,Triệu tập Hội Đồng Gió Mát - Tập 03,Đâu rồi hội đồng của những ngày xưa cũ,2020-01-20
5915631,Triệu tập Hội Đồng Gió Mát - Tập 03,Năm nay nghỉ TET dài dài chút nhẩy các lão.......,2020-01-31
5915632,Triệu tập Hội Đồng Gió Mát - Tập 03,Tổng múc thôi hội đồng,2020-01-31


## Load aggregated file for summarization

In [10]:
import pandas as pd
df = pd.read_csv("f319_data/aggregated_f319_data/aggregated_f319_posts.csv")


In [11]:
df

,topic_title,content,date
0,"Lạm phát và thị trường chứng khoán, mối liên h...","(Dữ liệu: thị trường Mỹ, 1926 - 2002. Hình từ ...",2025-10-26
1,"Lạm phát và thị trường chứng khoán, mối liên h...",Mới bơm tiền vài tháng. Ăn thua đ gì mà lo sớm...,2025-10-26
2,CỔ PHIẾU HLC – “VIÊN NGỌC ẨN” CỦA NGÀNH THAN,Công ty Cổ phần Than Hà Lầm – Vinacomin (\nHNX...,2025-10-23
3,CỔ PHIẾU HLC – “VIÊN NGỌC ẨN” CỦA NGÀNH THAN,"HLC quả rẻ bác, năm nay lãi to chia cổ tức tiề...",2025-10-24
4,CỔ PHIẾU HLC – “VIÊN NGỌC ẨN” CỦA NGÀNH THAN,"Vãi, sàn luôn.",2025-10-24
...,...,...,...
5457930,Triệu tập Hội Đồng Gió Mát - Tập 03,"Inbox gì mén\n--- Gộp bài viết,\n17/01/2020\n-...",2020-01-17
5457931,Triệu tập Hội Đồng Gió Mát - Tập 03,Đâu rồi hội đồng của những ngày xưa cũ,2020-01-20
5457932,Triệu tập Hội Đồng Gió Mát - Tập 03,Năm nay nghỉ TET dài dài chút nhẩy các lão.......,2020-01-31
5457933,Triệu tập Hội Đồng Gió Mát - Tập 03,Tổng múc thôi hội đồng,2020-01-31


## Get 100 most popular stock symbols in Fireant

In [13]:
stocks_df = pd.read_csv("fireant_data/sentiment_posts/sentiment_posts_count.csv")

# Count the total number of posts for each stock
post_counts = stocks_df.groupby('taggedSymbols').agg(
    total_posts=('total_posts', 'sum') 
).reset_index() 

# sort the stocks by total_posts in descending order
sorted_stocks = post_counts.sort_values(by='total_posts', ascending=False).reset_index(drop=True)
# get the list of 100 most mentioned stocks
top_100_stocks = sorted_stocks.head(100)['taggedSymbols'].tolist()
print(top_100_stocks)

['VNINDEX', 'DIG', 'VN30F1M', 'HPG', 'VND', 'SSI', 'NVL', 'CEO', 'VIX', 'DXG', 'VIC', 'HAG', 'HSG', 'SHB', 'GEX', 'VN30', 'PDR', 'NKG', 'SHS', 'BSR', 'MBB', 'STB', 'VHM', 'CII', 'FPT', 'KBC', 'MWG', 'HCM', '^DJI', 'BCG', 'POW', 'VCI', 'VPB', 'VCG', 'PVD', 'TCB', 'HQC', 'HHV', 'DGC', 'LDG', 'TCH', 'CTG', 'DCM', 'PVS', 'TPB', 'MSN', 'DBC', 'VRE', 'FLC', 'HAH', 'DPM', 'NLG', 'VNM', 'FTS', 'MSB', 'ACB', 'MBS', 'ITA', 'LCG', 'HDC', 'FRT', 'SCR', 'ORS', 'DGW', 'LPB', 'IDC', 'EIB', 'KHG', 'GVR', 'HNG', 'HBC', 'VOS', 'EVF', 'ANV', 'HUT', 'VCB', 'IDI', 'BID', 'HPX', 'HVN', 'SZC', 'PVT', 'VSC', 'VHC', 'CTD', 'HDG', 'ROS', 'TNG', 'ASM', 'APS', 'PC1', 'FCN', 'IDJ', 'FIT', 'GMD', 'DLG', '$BTC', 'VGI', 'VGC', 'CTS']


## Check Relations between Symbols and Topic Titles

In [14]:
import re

# create a regex pattern to match whole words only
pattern = r'\b(?:' + '|'.join(re.escape(s) for s in top_100_stocks) + r')\b'
regex = re.compile(pattern, re.IGNORECASE)

def extract_mentioned_symbols_regex(row):
    text = str(row['topic_title']) + ' ' + str(row['content'])
    matches = regex.findall(text)
    # normalize to uppercase and remove duplicates
    return list(set(match.upper() for match in matches))

# test function
extract_mentioned_symbols_regex(df.iloc[8921])

[]

In [15]:
df['mentionedSymbols'] = df.apply(extract_mentioned_symbols_regex, axis=1)

In [16]:
df = df[df['mentionedSymbols'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
df

,topic_title,content,date,mentionedSymbols
12,Chuẩn bị lên tầu đón siêu sóng Noel!,Sau 1 thời gian tạm nghỉ có nhắn nhủ AE tại\nh...,2025-10-26,"[VNINDEX, VCI]"
18,Chuẩn bị lên tầu đón siêu sóng Noel!,Nhóm P thì sao bác? GÁS PVS PVD?,2025-10-26,"[PVD, PVS]"
26,"Cổ phiếu sạch nhất ngành BĐS nhưng sau 8 năm, ...",Mình đã viết những dòng nhận xét\ncách đây 8 n...,2025-10-26,[NLG]
32,"MBB, Bank dẫn sóng....!",Hiệu quả – Bền vững – Tiên phong: Dấu ấn MB 9 ...,2025-10-26,[MBB]
33,"MBB, Bank dẫn sóng....!",Quá rẻ so gới giá trị thật,2025-10-26,[MBB]
...,...,...,...,...
5457873,Tuyển tập cổ phiếu có thể tăng >100% khi ra ti...,Ctd mua lệnh quét không cần kê lệnh. Tay to hố...,2020-01-22,[CTD]
5457885,Triệu tập Hội Đồng Gió Mát - Tập 03,"Uh, hồi xưa em chỉ múc pvd\n, giờ chuyển cv mú...",2020-01-02,[PVD]
5457893,Triệu tập Hội Đồng Gió Mát - Tập 03,Ngon cc. Cuối năm chả hú rộn cả lên múc CTG cò...,2020-01-02,[CTG]
5457914,Triệu tập Hội Đồng Gió Mát - Tập 03,"lão nói vậy không hẳn đúng đâu, ôm NKG là có c...",2020-01-16,[NKG]


In [17]:
# Save for future use
df.to_csv('f319_data/aggregated_f319_data/aggregated_f319_posts_with_symbols.csv', index=False)

In [18]:
df.iloc[1]['content']

'Nhóm P thì sao bác? GÁS PVS PVD?'

## Posts Cleaning

In [19]:
import re

def remove_links(text):
    # Xóa link dạng http(s)://... hoặc www....
    text = re.sub(r"http\S+", "", text)     # remove http:// hoặc https://
    text = re.sub(r"www\.\S+", "", text)    # remove www...
    text = re.sub(r"\S+\.com\S*", "", text) # remove .com/.net/.vn...
    return text.strip()

def clean_text(text):
    text = str(text).lower()                          # lowercase
    text = re.sub(r"\n+", ". ", text)                # replace new line with period
    text = remove_links(text)                         # remove links
    text = re.sub(r"@\w+", "", text)                  # remove mentions (@abc)
    text = re.sub(r"#\w+", "", text)                  # remove hashtags
    text = re.sub(
        r"[^0-9a-zA-Záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệ"
        r"íìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữự"
        r"ýỳỷỹỵđ\s!?,.]+", 
        " ", 
        text
    )            # keep only letters, numbers and some punctuation
    text = re.sub(r"\s+", " ", text).strip()          # remove extra spaces
    return text



In [20]:
df['content'] = df['content'].apply(clean_text)
df

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1768\1649223096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].apply(clean_text)


,topic_title,content,date,mentionedSymbols
12,Chuẩn bị lên tầu đón siêu sóng Noel!,sau 1 thời gian tạm nghỉ có nhắn nhủ ae tại. t...,2025-10-26,"[VNINDEX, VCI]"
18,Chuẩn bị lên tầu đón siêu sóng Noel!,nhóm p thì sao bác? gás pvs pvd?,2025-10-26,"[PVD, PVS]"
26,"Cổ phiếu sạch nhất ngành BĐS nhưng sau 8 năm, ...",mình đã viết những dòng nhận xét. cách đây 8 n...,2025-10-26,[NLG]
32,"MBB, Bank dẫn sóng....!",hiệu quả bền vững tiên phong dấu ấn mb 9 tháng...,2025-10-26,[MBB]
33,"MBB, Bank dẫn sóng....!",quá rẻ so gới giá trị thật,2025-10-26,[MBB]
...,...,...,...,...
5457873,Tuyển tập cổ phiếu có thể tăng >100% khi ra ti...,ctd mua lệnh quét không cần kê lệnh. tay to hố...,2020-01-22,[CTD]
5457885,Triệu tập Hội Đồng Gió Mát - Tập 03,"uh, hồi xưa em chỉ múc pvd. , giờ chuyển cv mú...",2020-01-02,[PVD]
5457893,Triệu tập Hội Đồng Gió Mát - Tập 03,ngon cc. cuối năm chả hú rộn cả lên múc ctg cò...,2020-01-02,[CTG]
5457914,Triệu tập Hội Đồng Gió Mát - Tập 03,"lão nói vậy không hẳn đúng đâu, ôm nkg là có c...",2020-01-16,[NKG]


In [21]:
# remove duplicates in content column
print(f"Number of posts before removing duplicates: {len(df)}")
df = df.drop_duplicates(subset=['content'])
print(f"Number of posts after removing duplicates: {len(df)}")

# reset index 
df = df.reset_index(drop=True)
df

Number of posts before removing duplicates: 2163562
Number of posts after removing duplicates: 2076873


,topic_title,content,date,mentionedSymbols
0,Chuẩn bị lên tầu đón siêu sóng Noel!,sau 1 thời gian tạm nghỉ có nhắn nhủ ae tại. t...,2025-10-26,"[VNINDEX, VCI]"
1,Chuẩn bị lên tầu đón siêu sóng Noel!,nhóm p thì sao bác? gás pvs pvd?,2025-10-26,"[PVD, PVS]"
2,"Cổ phiếu sạch nhất ngành BĐS nhưng sau 8 năm, ...",mình đã viết những dòng nhận xét. cách đây 8 n...,2025-10-26,[NLG]
3,"MBB, Bank dẫn sóng....!",hiệu quả bền vững tiên phong dấu ấn mb 9 tháng...,2025-10-26,[MBB]
4,"MBB, Bank dẫn sóng....!",quá rẻ so gới giá trị thật,2025-10-26,[MBB]
...,...,...,...,...
2076868,Tuyển tập cổ phiếu có thể tăng >100% khi ra ti...,ctd mua lệnh quét không cần kê lệnh. tay to hố...,2020-01-22,[CTD]
2076869,Triệu tập Hội Đồng Gió Mát - Tập 03,"uh, hồi xưa em chỉ múc pvd. , giờ chuyển cv mú...",2020-01-02,[PVD]
2076870,Triệu tập Hội Đồng Gió Mát - Tập 03,ngon cc. cuối năm chả hú rộn cả lên múc ctg cò...,2020-01-02,[CTG]
2076871,Triệu tập Hội Đồng Gió Mát - Tập 03,"lão nói vậy không hẳn đúng đâu, ôm nkg là có c...",2020-01-16,[NKG]


In [22]:
df.iloc[1]['content']

'nhóm p thì sao bác? gás pvs pvd?'

## Sentiment Analysis

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

# Load model
model_path = "Sentiment Model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Automatically use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


# ------------------------------------
#  Single text prediction
# ------------------------------------
def predict_sentiment(text):
    inputs = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=256,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(logits, dim=1).item()

    return pred

def predict_batch(text_list, batch_size=32):
    preds = []
    
    for i in tqdm(range(0, len(text_list), batch_size), desc="Predicting"):
        batch = text_list[i : i + batch_size]

        inputs = tokenizer(
            batch,
            truncation=True,
            padding=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits
            batch_preds = torch.argmax(logits, dim=1).cpu().tolist()
            preds.extend(batch_preds)

    return preds


In [36]:
# Convert the column to a list
texts = df["content"].astype(str).tolist()

# Run batch prediction
preds = predict_batch(texts, batch_size=32)

# Assign results back to df
df["sentiment"] = preds

Predicting: 100%|██████████| 64903/64903 [7:28:25<00:00,  2.41it/s]      


In [44]:
# count number rows with sentiment = 0
df[df['sentiment'] == 1].head(30)

,topic_title,content,date,mentionedSymbols,sentiment
0,Chuẩn bị lên tầu đón siêu sóng Noel!,sau 1 thời gian tạm nghỉ có nhắn nhủ ae tại. t...,2025-10-26,"[VNINDEX, VCI]",1
1,Chuẩn bị lên tầu đón siêu sóng Noel!,nhóm p thì sao bác? gás pvs pvd?,2025-10-26,"[PVD, PVS]",1
2,"Cổ phiếu sạch nhất ngành BĐS nhưng sau 8 năm, ...",mình đã viết những dòng nhận xét. cách đây 8 n...,2025-10-26,[NLG],1
3,"MBB, Bank dẫn sóng....!",hiệu quả bền vững tiên phong dấu ấn mb 9 tháng...,2025-10-26,[MBB],1
4,"MBB, Bank dẫn sóng....!",quá rẻ so gới giá trị thật,2025-10-26,[MBB],1
5,"MBB, Bank dẫn sóng....!","giá cao quá, tây bán rát, về thấp nữa đi",2025-10-26,[MBB],1
6,"MBB, Bank dẫn sóng....!",ko bank nào phát triển mạnh mẽ bằng mbb đâu. c...,2025-10-26,[MBB],1
8,"MBB, Bank dẫn sóng....!","ai đã bán mbb giá 27 28k, canh mua lại giá qua...",2025-10-26,[MBB],1
9,"MBB, Bank dẫn sóng....!",mbb ngon quá ae ơi,2025-10-26,[MBB],1
10,"MBB, Bank dẫn sóng....!",lợi nhuận tăng dc 12 nhưng giá đã tăng mịa nó ...,2025-10-26,[MBB],1


In [37]:
df.to_csv('f319_data/aggregated_f319_data/aggregated_f319_posts_with_symbols_and_sentiment.csv', index=False)

In [2]:
import pandas as pd
df = pd.read_csv('f319_data/aggregated_f319_data/aggregated_f319_posts_with_symbols_and_sentiment.csv')
df

,topic_title,content,date,mentionedSymbols,sentiment
0,Chuẩn bị lên tầu đón siêu sóng Noel!,sau 1 thời gian tạm nghỉ có nhắn nhủ ae tại. t...,2025-10-26,"['VNINDEX', 'VCI']",1
1,Chuẩn bị lên tầu đón siêu sóng Noel!,nhóm p thì sao bác? gás pvs pvd?,2025-10-26,"['PVD', 'PVS']",1
2,"Cổ phiếu sạch nhất ngành BĐS nhưng sau 8 năm, ...",mình đã viết những dòng nhận xét. cách đây 8 n...,2025-10-26,['NLG'],1
3,"MBB, Bank dẫn sóng....!",hiệu quả bền vững tiên phong dấu ấn mb 9 tháng...,2025-10-26,['MBB'],1
4,"MBB, Bank dẫn sóng....!",quá rẻ so gới giá trị thật,2025-10-26,['MBB'],1
...,...,...,...,...,...
2076868,Tuyển tập cổ phiếu có thể tăng >100% khi ra ti...,ctd mua lệnh quét không cần kê lệnh. tay to hố...,2020-01-22,['CTD'],1
2076869,Triệu tập Hội Đồng Gió Mát - Tập 03,"uh, hồi xưa em chỉ múc pvd. , giờ chuyển cv mú...",2020-01-02,['PVD'],1
2076870,Triệu tập Hội Đồng Gió Mát - Tập 03,ngon cc. cuối năm chả hú rộn cả lên múc ctg cò...,2020-01-02,['CTG'],1
2076871,Triệu tập Hội Đồng Gió Mát - Tập 03,"lão nói vậy không hẳn đúng đâu, ôm nkg là có c...",2020-01-16,['NKG'],0


In [5]:
# drop columns 'topic_title' and 'content'
df = df.drop(columns=['topic_title', 'content'])
df

,date,mentionedSymbols,sentiment
0,2025-10-26,"['VNINDEX', 'VCI']",1
1,2025-10-26,"['PVD', 'PVS']",1
2,2025-10-26,['NLG'],1
3,2025-10-26,['MBB'],1
4,2025-10-26,['MBB'],1
...,...,...,...
2076868,2020-01-22,['CTD'],1
2076869,2020-01-02,['PVD'],1
2076870,2020-01-02,['CTG'],1
2076871,2020-01-16,['NKG'],0


In [6]:
# Explode the 'mentionedSymbols' column
df['mentionedSymbols'] = df['mentionedSymbols'].apply(lambda x: eval(x) if isinstance(x, str) else [])
df = df.explode('mentionedSymbols').reset_index(drop=True)
df

,date,mentionedSymbols,sentiment
0,2025-10-26,VNINDEX,1
1,2025-10-26,VCI,1
2,2025-10-26,PVD,1
3,2025-10-26,PVS,1
4,2025-10-26,NLG,1
...,...,...,...
2955196,2020-01-22,CTD,1
2955197,2020-01-02,PVD,1
2955198,2020-01-02,CTG,1
2955199,2020-01-16,NKG,0


In [7]:
# group by date and mentionedSymbols
df_grouped = df.groupby(['date', 'mentionedSymbols']).agg(
    total_posts=('sentiment', 'count'),
    positive_posts=('sentiment', lambda x: (x == 1).sum()),
    negative_posts=('sentiment', lambda x: (x == 0).sum()),
).reset_index()

df_grouped

,date,mentionedSymbols,total_posts,positive_posts,negative_posts
0,2020-01-02,ACB,3,1,2
1,2020-01-02,ASM,2,1,1
2,2020-01-02,BCG,2,0,2
3,2020-01-02,BID,42,33,9
4,2020-01-02,BSR,3,3,0
...,...,...,...,...,...
168041,2025-10-26,VN30,4,4,0
168042,2025-10-26,VND,9,8,1
168043,2025-10-26,VNINDEX,17,16,1
168044,2025-10-26,VNM,1,1,0


In [8]:
df_grouped.to_csv('f319_data/aggregated_f319_data/sentiment_summary_by_stock_and_date.csv', index=False)